In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
import panel as pn
import os

pn.extension("plotly")

In [2]:
# Variáveis do arquivo .env
load_dotenv()

True

In [3]:
# Conexão com o Banco de Dados
usuario = os.getenv("DB_USER")
senha = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
nome_do_banco = os.getenv("DB_NAME")

engine = create_engine(
    f"postgresql+psycopg2://{usuario}:{senha}@{host}:{port}/{nome_do_banco}"
)

In [ ]:
# Receita, despesa, lucro dos Serviços - Total
# Tabelas Serviço e Financeiro
servico = pd.read_sql("servico", con=engine)
financeiro = pd.read_sql("financeiro", con=engine)
# JOIN nas duas tabelas
servico_lucro_prejuizo = pd.merge(servico, financeiro, how="inner", on="servico_id")
# Agrupar os dados por serviço para métricas, depois ordenar por lucro de forma crescente
agrupado = (
    servico_lucro_prejuizo.groupby("nome_servico")
    .agg({
        "receita": "sum",
        "despesa": "sum",
        "lucro": "sum",
    })
    .sort_values(by="lucro", ascending=True)
    .reset_index()
)

print(agrupado.head())


              nome_servico     receita     despesa      lucro
0                     Tech  1081824.72   846942.51  443696.96
1  Planejamento Tributário  1320170.29  1004377.70  511106.11
2     Consultoria Contábil  1227489.57   910938.80  554376.20
3    Assessoria Financeira  1333229.73  1007058.42  555473.99
4            Contabilidade  1536826.51   991085.04  767215.39


In [73]:
# Gráfico Comparação de Receita, Despesa e Lucro por Serviço

fig = px.bar(
    agrupado,
    x="nome_servico",
    y=["receita", "despesa", "lucro"],
    title="Comparação de Receita, Despesa e Lucro por Serviço",
    labels={"value": "Valor (R$)", "nome_servico": "Serviço", "variable": "Variáveis"},
    barmode="group",
    width=1200,
    height=600,
    template="seaborn",
)

fig.show()
